In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

# Area de estudio

Debido al tamaño del stack de bandas es probable que el tamaño del area de estudio sea demasiado grande para exportar los datos, por lo que una alternativa es seleccionar puntos aleatorios dentro de la misma y generar un buffer sobre ellos para obetner una muestra representativa.

In [2]:
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/mdg_south')
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

year = 2022
start = f'{year}-01-01'
end = f'{2023}-03-01'

# Llamado a las colecciones

### Optico

In [11]:
l8_col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUD_COVER', 30))

def mask_clouds_landsat(image):
    qa = image.select('QA_PIXEL')
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)


l8_masked = l8_col.map(mask_clouds_landsat)
image = l8_masked.median().toFloat().clip(roi)

ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')  # NIR y Red
mndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')  # Green y SWIR1
ndbi = image.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')  # SWIR1 y NIR
nbr = image.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR')  # NIR y SWIR2
ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')

rvi_opt = image.expression(
    '(NIR / RED)',
    {
        'NIR': image.select('SR_B5'),
        'RED': image.select('SR_B4').where(image.select('SR_B4').eq(0), 0.0001)
    }
).rename('RVI')

evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': image.select('SR_B5'),  # NIR
        'RED': image.select('SR_B4'),  # Red
        'BLUE': image.select('SR_B2')  # Blue
    }
).rename('EVI')

savi = image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': image.select('SR_B5'),
        'RED': image.select('SR_B4')
    }
).rename('SAVI')

cvi = image.expression(
    'NIR * (RED / GREEN)',
    {
        'NIR': image.select('SR_B5'),
        'RED': image.select('SR_B4'),
        'GREEN': image.select('SR_B3')
    }
).rename('CVI')

image = image.addBands([ndvi, mndwi, ndbi, evi, savi, rvi_opt, nbr, ndwi, cvi])

# Dynamic World collection for label extraction
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

# Get the median label from Dynamic World
label = dw_col.select('label').median().rename('label')

# Add DEM, slope, and aspect from SRTM DEM
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)
aspect = ee.Terrain.aspect(dem).rename('aspect').clip(roi)

# Add elevation, slope, aspect, and label to the image
image = image.addBands([dem.rename('elevation'), slope, aspect, label])

# Palsar


In [ ]:
import math

def terrain_correction_palsar(image, roi, angle_image):

    srtm = ee.Image('USGS/SRTMGL1_003').clip(roi)    
    # Convertir la imagen a dB usando la fórmula γ₀ = 10 log₁₀(DN²) - 83.0 dB
    dn_squared = image.pow(2)   
    backscatter = ee.Image.constant(10).multiply(dn_squared.log10()).subtract(83.0)
    theta_i = angle_image  
    
    # Obtener el aspecto promedio de la región de interés
    phi_i = ee.Terrain.aspect(theta_i).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,  # Usar la ROI
        scale=1000
    ).get('aspect')
    
    # Convertir phi_i a una imagen constante
    phi_i_image = ee.Image.constant(phi_i)
    
    # Geometría del terreno
    alpha_s = ee.Terrain.slope(srtm).select('slope')
    phi_s = ee.Terrain.aspect(srtm).select('aspect')
    
    # Geometría del modelo
    phi_r = phi_i_image.subtract(phi_s)
    
    # Convertir todo a radianes
    phi_r_rad = phi_r.multiply(math.pi / 180)
    alpha_s_rad = alpha_s.multiply(math.pi / 180)
    theta_i_rad = theta_i.multiply(math.pi / 180)
    ninety_rad = ee.Image.constant(90).multiply(math.pi / 180)

    # Pendiente en el rango
    alpha_r = (alpha_s_rad.tan().multiply(phi_r_rad.cos())).atan()

    # Pendiente en acimut
    alpha_az = (alpha_s_rad.tan().multiply(phi_r_rad.sin())).atan()

    # Ángulo de incidencia local
    theta_lia = (alpha_az.cos().multiply((theta_i_rad.subtract(alpha_r)).cos())).acos()
    theta_lia_deg = theta_lia.multiply(180 / math.pi)

    # Gamma_nought_flat usando la fórmula ajustada
    gamma0 = backscatter.divide(theta_i_rad.cos())
    gamma0_dB = gamma0.rename('gamma0_dB')

    # Modelo volumétrico
    nominator = (ninety_rad.subtract(theta_i_rad).add(alpha_r)).tan()
    denominator = (ninety_rad.subtract(theta_i_rad)).tan()
    vol_model = (nominator.divide(denominator)).abs()

    # Aplicar el modelo
    gamma0_volume = gamma0.divide(vol_model)
    gamma0_volume_dB = gamma0_volume.rename('gamma0_volume_dB')

    return gamma0_dB


def preprocess(image, roi, angle_image):
    image = terrain_correction_palsar(image, roi, angle_image)
    speckle_filtered = image.focal_median(kernelType='circle', radius=50, units='meters')
    return speckle_filtered


angle = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('angle')  
    .median() 
)

img_hh = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HH')  # Seleccionar solo HH
    .map(lambda img: preprocess(img, roi, angle))  # Pasar roi y angle_image al preprocesamiento
)

img_hv = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HV')  # Seleccionar solo HV
    .map(lambda img: preprocess(img, roi, angle))  # Pasar roi y angle_image al preprocesamiento
)

palsar_hh_median = img_hh.median().rename('HH_dB')
palsar_hv_median = img_hv.median().rename('HV_dB')

hh_plus_hv = palsar_hh_median.add(palsar_hv_median).rename('HH_plus_HV')
hh_minus_hv = palsar_hh_median.subtract(palsar_hv_median).rename('HH_minus_HV')
hh_div_hv = palsar_hh_median.divide(palsar_hv_median).rename('HH_div_HV')
rvi = palsar_hv_median.divide(palsar_hh_median.add(palsar_hv_median)).multiply(4).rename('RVI_palsar')

palsar_combined = ee.Image.cat([palsar_hh_median, palsar_hv_median, hh_plus_hv, hh_minus_hv, hh_div_hv, rvi]).clip(roi)
image = image.addBands(palsar_combined)


def calculate_texture_metrics(image, band_name, roi, size=5):
    
    band_int = image.select(band_name).toInt32().clip(roi)
    glcm = band_int.glcmTexture(size=size)
    
    # Extraer las texturas de interés y renombrarlas
    contrast = glcm.select(f'{band_name}_contrast').rename(f'{band_name}_Contrast')
    correlation = glcm.select(f'{band_name}_corr').rename(f'{band_name}_Correlation')
    entropy = glcm.select(f'{band_name}_ent').rename(f'{band_name}_Entropy')
    inertia = glcm.select(f'{band_name}_inertia').rename(f'{band_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])

image = calculate_texture_metrics(image, 'HH_dB', roi)
image = calculate_texture_metrics(image, 'HV_dB', roi)

bands = [
    'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6',  # Landsat 8 RGB, NIR, SWIR1
    'MNDWI', 'NDBI', 'NDVI', 'EVI', 'SAVI','NDWI','CVI', # Indices
    'elevation', 'slope', 'aspect','label',  # Topographic bands
    'HH_dB', 'HV_dB',  # PALSAR backscatter
    'HH_plus_HV', 'HH_minus_HV', 'HH_div_HV', 'RVI_palsar',  # PALSAR-based indices
    'HH_dB_Contrast', 'HH_dB_Correlation', 'HH_dB_Entropy', 'HH_dB_Inertia',  # PALSAR texture metrics
    'HV_dB_Contrast', 'HV_dB_Correlation', 'HV_dB_Entropy', 'HV_dB_Inertia', 
    'RVI', 'NBR'  # Indices related to Landsat and PALSAR
]

image = image.select(bands)

In [5]:
def calculate_texture_metrics(image, band_name, roi, size=5):
    
    band_int = image.select(band_name).toInt32().clip(roi)
    glcm = band_int.glcmTexture(size=size)
    
    # Extraer las texturas de interés y renombrarlas
    contrast = glcm.select(f'{band_name}_contrast').rename(f'{band_name}_Contrast')
    correlation = glcm.select(f'{band_name}_corr').rename(f'{band_name}_Correlation')
    entropy = glcm.select(f'{band_name}_ent').rename(f'{band_name}_Entropy')
    inertia = glcm.select(f'{band_name}_inertia').rename(f'{band_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])

image = calculate_texture_metrics(image, 'HH_dB', roi)
image = calculate_texture_metrics(image, 'HV_dB', roi)

In [ ]:
import ee, math

# ────────────────────────────────────────────────────────────────
# 1. CONSTANTES
# ────────────────────────────────────────────────────────────────
ALOS_CALIB_CONST_DB = -83.0                         # JAXA metadata
PI_OVER_2_IMG       = ee.Image.constant(math.pi/2)  # π/2 en formato Image

# ────────────────────────────────────────────────────────────────
# 2. FUNCIONES AUXILIARES
# ────────────────────────────────────────────────────────────────
def to_sigma0(image):
    """DN → σ⁰ (lineal) y lo devuelve como ee.Image."""
    sigma0 = image.pow(2).multiply(10 ** (ALOS_CALIB_CONST_DB / 10))
    sigma0 = ee.Image(sigma0)                        # asegura tipo Image
    sigma0 = ee.Image(sigma0.copyProperties(image, image.propertyNames()))
    return sigma0

def refined_lee(img):
    """Filtro Refined-Lee placeholder (sustituye por cuerpo real)."""
    return img       # ← pon aquí tu implementación completa

def get_aw3d30_dem(roi):
    """Mosaico DEM AW3D30 recortado al ROI, banda 'DSM'."""
    dem_ic = (ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')
                .filterBounds(roi)
                .select('DSM'))
    dem = (dem_ic.mosaic()
                   .clip(roi)
                   .setDefaultProjection('EPSG:4326', None, 30))
    return dem

def rtc_small2011(sigma0_img, angle_img, dem, model='volumetric'):
    """
    Radiometric Terrain Flattening — Small (2011).
    Retorna γ⁰ lineal corregido por topografía.
    """
    terr        = ee.Terrain.products(dem)
    slope_rad   = terr.select('slope').multiply(math.pi/180)
    aspect_rad  = terr.select('aspect').multiply(math.pi/180)

    theta_i_rad = angle_img.multiply(math.pi/180)
    phi_r       = aspect_rad.subtract(PI_OVER_2_IMG)

    alpha_r = slope_rad.tan().multiply(phi_r.cos()).atan()
    alpha_az= slope_rad.tan().multiply(phi_r.sin()).atan()
    theta_lia = alpha_az.cos().multiply(theta_i_rad.subtract(alpha_r).cos()).acos()

    if model == 'volumetric':
        k = (PI_OVER_2_IMG.subtract(theta_i_rad).add(alpha_r).tan()
             .divide(PI_OVER_2_IMG.subtract(theta_i_rad).tan())).abs()
    else:
        k = theta_i_rad.subtract(alpha_r).cos().divide(theta_i_rad.cos())

    gamma0      = sigma0_img.divide(ee.Image(theta_i_rad.cos()))
    gamma0_flat = gamma0.divide(k)

    return gamma0_flat.updateMask(theta_lia.lt(PI_OVER_2_IMG))

# ────────────────────────────────────────────────────────────────
# 3. PREPROCESAMIENTO PALSAR
# ────────────────────────────────────────────────────────────────
def preprocess_palsar(img, roi):
    """
    Devuelve HH_norm y HV_norm en dB listos para añadir al stack.
    """
    dem    = get_aw3d30_dem(roi)

    angle  = img.select('angle')          # ángulo de incidencia (°)
    sigma0 = to_sigma0(img.select(['HH', 'HV']))

    # 3.1  Terrain-flattening (lineal) y paso a dB
    rtc_lin = rtc_small2011(sigma0, angle, dem)
    rtc_db  = rtc_lin.log10().multiply(10)

    # 3.2  Speckle filtering
    clean = refined_lee(rtc_db)

    # 3.3  Detrend por ángulo de incidencia (por escena y por polarización)
    norm_bands = []
    for pol in ['HH', 'HV']:
        band = clean.select(pol)              # y
        xy   = ee.Image.cat([angle.rename('x'), band.rename('y')])
        fit  = xy.reduceRegion(ee.Reducer.linearFit(),
                            geometry=roi,
                            scale=90,
                            bestEffort=True)
        slope     = ee.Number(fit.get('scale'))
        intercept = ee.Number(fit.get('offset'))
        norm      = band.subtract(angle.multiply(slope)).subtract(intercept) \
                        .rename(f'{pol}_norm')
        norm_bands.append(norm)

    return ee.Image.cat(norm_bands)  # → HH_norm, HV_norm (dB)

# ────────────────────────────────────────────────────────────────
# 4. APLICAR A LA COLECCIÓN
# ────────────────────────────────────────────────────────────────
hh_hv_clean = (ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
                 .filterBounds(roi)
                 .filterDate(start, end)
                 .map(lambda im: preprocess_palsar(im, roi))
                 .median())

palsar_hh = hh_hv_clean.select('HH_norm').rename('HH_dB')
palsar_hv = hh_hv_clean.select('HV_norm').rename('HV_dB')
palsar_combined = ee.Image.cat([palsar_hh, palsar_hv]).clip(roi)


# ────────────────────────────────────────────────────────────────
# 4-bis.  Radar-based indices (linear → ratios → back to Image)
# ────────────────────────────────────────────────────────────────
TEN = ee.Image.constant(10)

# 2. dB → lineal   (σ = 10^(dB/10))
hh_lin = TEN.pow(palsar_hh.divide(10))
hv_lin = TEN.pow(palsar_hv.divide(10))

# 3. Índices lineales
hh_plus_hv  = hh_lin.add(hv_lin).rename('HH_plus_HV')
hh_minus_hv = hh_lin.subtract(hv_lin).rename('HH_minus_HV')
hh_div_hv   = hh_lin.divide(hv_lin).rename('HH_div_HV')
rvi         = hv_lin.multiply(4).divide(hh_lin.add(hv_lin)).rename('RVI_palsar')

# 4. Concatenar radar + índices
palsar_combined = ee.Image.cat([
    palsar_hh, palsar_hv,
    hh_plus_hv, hh_minus_hv, hh_div_hv, rvi
]).clip(roi)

# 5. Añadir al stack existente
image = image.addBands(palsar_combined)

print('Bandas finales:', image.bandNames().getInfo())


EEException: reduce.median: Error in map(ID=2022):
Image.reduceRegion: Too many pixels in the region. Found 62901939358, but maxPixels allows only 10000000.
Ensure that you are not aggregating at a higher resolution than you intended; that is a frequent cause of this error. If not, then you may set the 'maxPixels' argument to a limit suitable for your computation; set 'bestEffort' to true to aggregate at whatever scale results in 'maxPixels' total pixels; or both.

In [53]:
Map.centerObject(roi, 10)
Map.addLayer(image.select('HH_dB'), {}, 'HH_dB', False)
# Map.addLayer(image.select('HV_dB'), {}, 'HV_dB', False)

Map

Map(center=[-24.80782264166506, 46.9413487442998], controls=(WidgetControl(options=['position', 'transparent_b…

In [59]:
bands = [
    'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6',  # Landsat 8 RGB, NIR, SWIR1
    'MNDWI', 'NDBI', 'NDVI', 'EVI', 'SAVI','NDWI','CVI', # Indices
    'elevation', 'slope', 'aspect','label',  # Topographic bands
    'HH_dB', 'HV_dB',  # PALSAR backscatter
    'HH_plus_HV', 'HH_minus_HV', 'HH_div_HV', 'RVI_palsar',  # PALSAR-based indices
    'RVI' # Indices related to Landsat and PALSAR
]

image = image.select(bands)

In [60]:
print(image.bandNames().getInfo())

['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'MNDWI', 'NDBI', 'NDVI', 'EVI', 'SAVI', 'NDWI', 'CVI', 'elevation', 'slope', 'aspect', 'label', 'HH_dB', 'HV_dB', 'HH_plus_HV', 'HH_minus_HV', 'HH_div_HV', 'RVI_palsar', 'RVI']


# Muestreo de datos

En esta seccion se genera la logica de llamado a los datos de GEDI y recoleccion de datos sobre sus pixeles, la exportacion del csv se gerena por partes para cada uno de los buffers de puntos creados anteriormente.  

In [61]:
def mask_based_on_label_values(image, values):
    # Cargar la colección de DynamicWorld
    dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .select('label') \
        .median() \
        .clip(roi)
    
    # Crear una máscara basada en los valores de la lista
    label_mask = dynamic_world.eq(values[0])  # Inicializar con el primer valor
    for value in values[1:]:
        label_mask = label_mask.Or(dynamic_world.eq(value))  # Añadir los demás valores
    
    # Aplicar la máscara de valores al conjunto de datos
    return image.updateMask(label_mask)

# Ejemplo: filtrar GEDI según árboles (1), agua (5), y vegetación alta (2)
values_to_mask = [0,1,2,3,4,5,6,7,8] 


def get_gedi_data(band_name):
    return ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .map(lambda image: image.updateMask(
            image.select('degrade_flag').eq(0)  # Filtrar donde degrade_flag es 0
            .And(image.select('l2_quality_flag').eq(1))  # Filtrar donde l2_quality_flag es 1
            .And(image.select('l4_quality_flag').eq(1))  # Filtrar donde l4_quality_flag es 1
        )) \
        .select(band_name) \
        .median() \
        .toFloat() \
        .clip(roi)

# Obtener las bandas agbd, agbd_se y l4_quality_flag
gediData_agbd = get_gedi_data('agbd').rename('agbd')
gediData_agbd_se = get_gedi_data('agbd_se').rename('agbd_se')
gedi_combined = ee.Image.cat([gediData_agbd, gediData_agbd_se])
gedi_masked = mask_based_on_label_values(gedi_combined, values_to_mask)

# Convertir a float si es necesario
def convert_to_float(image):
    return image.float()

# Convertir la imagen GEDI enmascarada a float
gedi_masked = convert_to_float(gedi_masked)

# Puedes agregar las bandas filtradas de GEDI a tu imagen original si lo necesitas
image = image.addBands(gedi_masked)

In [64]:
import time

sample = image.addBands(gediData_agbd).updateMask(gediData_agbd).sample(
        scale=100,
        region=roi,
        geometries=True
    )

export_task = ee.batch.Export.table.toDrive(
        collection=sample,
        description='ExportSampleToCSV',
        folder='nuevo_radar_gedi',
        fileNamePrefix=f'Datos_RF_2022',
        fileFormat='CSV'
    )

# Iniciar la tarea de exportación
export_task.start()

    # Esperar a que la tarea de exportación se complete
export_task.status()

    # Verificar el estado de la tarea y mostrar un mensaje de éxito
while export_task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente
    
    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación completada con éxito.')
    else:
        print(f'Error en la exportación: {export_task.status()}')

Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'priority': 100, 'creation_timestamp_ms': 1748892743129, 'update_timestamp_ms': 1748892773278, 'start_timestamp_ms': 1748892753060, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'batch_eecu_usage_seconds': 91.054641723, 'id': 'IFL5OZG2LS4CCW5ZAFVY7WKM', 'name': 'projects/ee-facuboladerasgee/operations/IFL5OZG2LS4CCW5ZAFVY7WKM'}
Exportación en progreso...
Exportación completada con éxito.


In [64]:
# import time

# # Definir la carpeta de destino en Google Drive
# drive_folder = 'EE_Kisangani_reduced'

# # Función para procesar y exportar cada celda
# def process_and_export_cell(cell):
#     # Obtener el ID de la celda
#     grid_id = cell.get('grid_id')
    
#     # Obtener la geometría de la celda
#     cell_geom = cell.geometry()
    
#     # Realizar la operación deseada (por ejemplo, muestreo)
#     sample = image.addBands(gediData_agbd).updateMask(gediData_agbd).sample(
#         scale=30,
#         region=cell_geom,
#         geometries=True
#     )
    
#     # Configurar la tarea de exportación
#     task = ee.batch.Export.table.toDrive(
#         collection=sample,
#         description=ee.String('ExportSampleToCSV_GridCell_').cat(grid_id).getInfo(),
#         folder=drive_folder,
#         fileNamePrefix=ee.String('Datos_RF_2020_GridCell_').cat(grid_id).getInfo(),
#         fileFormat='CSV'
#     )
    
#     # Iniciar la tarea de exportación
#     task.start()
    
#     print(f"Tarea de exportación para la celda de la cuadrícula {grid_id.getInfo()} iniciada.")
#     return task

# # Lista para mantener un seguimiento de las tareas
# tasks = []

# # Controlar el número máximo de tareas activas
# max_concurrent_tasks = 3

# # Función para obtener el número de tareas activas
# def get_active_task_count():
#     tasks_list = ee.data.listOperations()
#     running_tasks = [t for t in tasks_list if 'RUNNING' in t['metadata']['state']]
#     return len(running_tasks)

# # Iterar sobre las celdas y procesarlas
# for i in range(n_cells):
#     cell = ee.Feature(grid_list.get(i))
#     task = process_and_export_cell(cell)
#     tasks.append(task)
    
#     # Controlar el número de tareas activas
#     while get_active_task_count() >= max_concurrent_tasks:
#         print('Esperando a que se liberen tareas...')
#         time.sleep(30)  # Esperar 30 segundos antes de verificar nuevamente

# print("Todas las tareas de exportación han sido iniciadas.")

# # Monitorizar las tareas (opcional)
# while any([task.active() for task in tasks]):
#     print('Exportación en progreso...')
#     time.sleep(30)
#     for task in tasks:
#         status = task.status()
#         print(f"Tarea {task.config['description']}: {status['state']}")

# print("Todas las tareas de exportación han sido completadas.")


Tarea de exportación para la celda de la cuadrícula 26,0 iniciada.
Tarea de exportación para la celda de la cuadrícula 27,0 iniciada.
Tarea de exportación para la celda de la cuadrícula 28,0 iniciada.
Tarea de exportación para la celda de la cuadrícula 29,0 iniciada.
Tarea de exportación para la celda de la cuadrícula 25,1 iniciada.
Tarea de exportación para la celda de la cuadrícula 26,1 iniciada.
Esperando a que se liberen tareas...
Esperando a que se liberen tareas...
Esperando a que se liberen tareas...
Tarea de exportación para la celda de la cuadrícula 27,1 iniciada.
Tarea de exportación para la celda de la cuadrícula 28,1 iniciada.
Tarea de exportación para la celda de la cuadrícula 29,1 iniciada.
Esperando a que se liberen tareas...
Esperando a que se liberen tareas...
Esperando a que se liberen tareas...
Tarea de exportación para la celda de la cuadrícula 30,1 iniciada.
Tarea de exportación para la celda de la cuadrícula 23,2 iniciada.
Tarea de exportación para la celda de la 